In [1]:
# we verify if the uncertainty points selected by active learning strategy is indeed lying on decision boundary
# least confidence strategy

In [ ]:
from deepvisualinsight.MMS import MMS
from deepvisualinsight import utils
import sys
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import json
import tensorflow as tf

In [ ]:
content_path = "E:\\DVI_exp_data\\active_learning\\LeastConfidence"
sys.path.append(content_path)

from Model.model import *
net = ResNet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 1, 20, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

In [ ]:
# hyperparameters
EPOCH = 10

In [ ]:
# 1. whether selected points lie on boundary?
new_index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH+1), "index.json")
new_index = utils.load_labelled_data_index(new_index_file)
training_data = mms.training_data[new_index]
training_labels = mms.training_labels[new_index].cpu().numpy()
train_data = mms.get_representation_data(EPOCH, training_data)

index_file = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH), "index.json")
index = utils.load_labelled_data_index(index_file)
l = []
for j in range(len(new_index)):
    if new_index[j] not in index:
        l.append(j)

preds = mms.get_pred(EPOCH, train_data)
preds = preds + 1e-8

sort_preds = np.sort(preds)
diff = (sort_preds[:, -1] - sort_preds[:, -2]) / (sort_preds[:, -1] - sort_preds[:, 0])
border = np.zeros(len(diff), dtype=np.bool)
border[diff < 0.15] = 1

# how many of them lie on boundary
s = np.sum(border)

In [ ]:
# 2. if point lies on boundary, is the background white?
grid_view, decision_view =mms.get_epoch_decision_view(EPOCH, resolution=100)
grid_view = grid_view.reshape(-1, 2)
decision_view = decision_view.reshape(-1,3)
from sklearn.neighbors import NearestNeighbors
nbs = NearestNeighbors(n_neighbors=1, algorithm="ball_tree").fit(grid_view)

target_samples = train_data[border]
repr_2d = mms.batch_project(target_samples, EPOCH)
_, indices = nbs.kneighbors(repr_2d)
indices = indices.squeeze()
sample_colors = decision_view[indices]

c1 = np.zeros(indices.shape[0], dtype=np.bool)
c1[sample_colors[:,0]>0.85] = 1

c2 = np.zeros(indices.shape[0], dtype=np.bool)
c2[sample_colors[:,1]>0.85] = 1

c3 = np.zeros(indices.shape[0], dtype=np.bool)
c3[sample_colors[:,2]>0.85] = 1
c = np.logical_and(c1,c2)
c=np.logical_and(c, c3)

np.sum(c), len(c), np.sum(c)/len(c)

